# Cartoonizing an Image

In this chapter, we are going to learn how to convert an image into a cartoon-like image. We will learn how to access the webcam and take keyboard/mouse inputs during a live video stream. We will also learn about some advanced image filters and see how we can use them to cartoonize an image.

By the end of this chapter, you will know: 

• How to access the webcam 

• How to take keyboard and mouse inputs during a live video stream 

• How to create an interactive application 

• How to use advanced image filters 

• How to cartoonize an image 

# Accessing the webcam

We can build very interesting applications using the live video stream from the webcam. OpenCV provides a video capture object which handles everything related to opening and closing of the webcam. All we need to do is create that object and keep reading frames from it.

In [1]:
import cv2 

video = cv2.VideoCapture(0)

a = 0 

while True:
    
    check,frame = video.read()
    a=a+1
    
#    print(frame)
    cv2.imshow('webcam',frame)
    key = cv2.waitKey(1)
    if key ==ord('q'):
        cv2.destroyAllWindows()
        break

video.release()        




# Mouse inputs

In this section, we will see how to use the mouse to interact with the display window. Let's start with something simple. We will write a program that will detect the quadrant in which the mouse click was detected. Once we detect it, we will highlight that quadrant.

In [2]:
def detect_quadrant(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN:
        if x > width/2:
            if y > height/2:
                point_top_left = (int(width/2),int(height/2))
                point_bottom_right = (width-1,height-1)
                
            else :
                point_top_left = (int(width/2),0)
                point_bottom_right = (width-1,int(height/2))
        else:           
            if y > height/2:             
                 point_top_left = (0, int(height/2))   
                 point_bottom_right = (int(width/2), height-1)      
            else:           
                point_top_left = (0, 0)          
                point_bottom_right = (int(width/2), int(height/2))
        cv2.rectangle(img, (0,0), (width-1,height-1), (255,255,255), -1)     
        cv2.rectangle(img, point_top_left, point_bottom_right, (0,100,0), -1)

In [3]:
import cv2 
import numpy as np 

# parameter of image 
width, height = 640, 480

#create image 
img = 255 * np.ones((height, width, 3), dtype=np.uint8)


cv2.namedWindow('Input window') 
cv2.setMouseCallback('Input window', detect_quadrant)
while True:
    cv2.imshow('Input window', img) 
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == ord('q'):
        break
cv2.destroyAllWindows()    

# Interacting with a live video stream 


Let's see how we can use the mouse to interact with live video stream from the webcam. We can use the mouse to select a region and then apply the "negative film" effect on that region, as shown next:

In [1]:
import cv2 
import numpy as np

def draw_rectangle(event, x, y, flags, params): 
    global x_init, y_init, drawing , top_left_pt , bottom_right_pt
    
    if event == cv2.EVENT_LBUTTONDOWN:    
        drawing = True     
        x_init, y_init = x, y
    elif event == cv2.EVENT_MOUSEMOVE:    
        if drawing:       
            top_left_pt = (min(x_init, x), min(y_init, y))     
            bottom_right_pt = (max(x_init, x), max(y_init, y))       
            img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x]
            
    elif event == cv2.EVENT_LBUTTONUP:    
        drawing = False       
        top_left_pt = (min(x_init, x), min(y_init, y))  
        bottom_right_pt = (max(x_init, x), max(y_init, y))       
        img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x]
        
if __name__=='__main__':   
    drawing = False   
    top_left_pt, bottom_right_pt = (-1,-1), (-1,-1)
    cap = cv2.VideoCapture(0)
    # Check if the webcam is opened correctly    
    if not cap.isOpened():      
        raise IOError("Cannot open webcam")
    cv2.namedWindow('Webcam') 
    cv2.setMouseCallback('Webcam', draw_rectangle)
    
    while True:   
        ret, frame = cap.read()   
        img = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)   
        (x0,y0), (x1,y1) = top_left_pt, bottom_right_pt       
        img[y0:y1, x0:x1] = 255 - img[y0:y1, x0:x1]  
        cv2.imshow('Webcam', img)
        c = cv2.waitKey(1)    
        if c == 27:           
            break
    cap.release()   
    cv2.destroyAllWindows()


# cartonize the image 

In [ ]:
import cv2
import numpy as np 
def cartonizing_image(img,ds_factro=4,sketch_mode=False):
    #convert img to gray scale 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    #applay median filter to gray scale
    img_gray = cv2.medianBlur(gray,7)
    
    #Detect edge in the image and threshold it 
    edges = cv2.Laplacian(img_gray,cv2.CV_8U,ksize=5)
    ret , mask = cv2.threshold(edges,100,255,cv2.THRESH_TRUNC)
    
    if sketch_mode:
        return cv2.cvtColor(mask,cv2.COLOR_GRAY2RGB)
    
if __name__=='__main__':
    cap = cv2.VideoCapture(0)
    cur_char = -1
    pre_char = -1
    while True :
        ret,frame = cap.read()
        frame = cv2.resize(frame,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
        c = cv2.waitKey(1)
        if c==27:
            break
        if c > -1 and c !=pre_char:
            cur_char = c
            pre_char = c
        if cur_char == ord('s'):
            cv2.imshow('cartonize',cartonizing_image(frame,sketch_mode=True))
        else:    
            cv2.imshow('Cartoonize', frame)    
            
    cap.release()
    cv2.destroyAllWindows()